In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import os
%matplotlib inline

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3 # This function returns a Keras image classification model (cnn formed by 7 layers)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense ,Flatten , BatchNormalization ,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img , img_to_array
inception_weights_path ='https://github.com/laxmimerit/male-female-face-dataset'


# Data preparation 

In [ ]:
!git clone https://github.com/laxmimerit/male-female-face-dataset.git

In [ ]:
epochs = 50 
lr= 1e-3
data= []
labels =[]
batch_size=128

In [ ]:
size =224 # height/width

# **Image Data Generator**

In [ ]:
train_datagen =  ImageDataGenerator(horizontal_flip=True,width_shift_range=0.4, height_shift_range=0.4,zoom_range=20,rescale=1/255)

In [ ]:
test_gen=ImageDataGenerator(rescale=1/255)

In [ ]:
target_size=(size,size)
target_size

In [ ]:
train_generator=train_datagen.flow_from_directory(directory='/content/male-female-face-dataset/Training',
                                  target_size=target_size,
                                  batch_size=batch_size,
                                  class_mode='binary') # because we 've 2 classes males and females 

In [ ]:
validation_generator=test_gen.flow_from_directory(directory='/content/male-female-face-dataset/Training',
                                  target_size=target_size,
                                  batch_size=batch_size,
                                  class_mode='binary') 

In [ ]:
train_generator.class_indices # the index of each class 

In [ ]:
train_generator.classes

In [ ]:
train_generator.class_mode # the nb of classes 

In [ ]:
x,y=train_generator.next()
x.shape #(nb of imgs ,lengh,width, red green blue)

# Build ML **Model**

In [ ]:
model = Sequential()
model.add(InceptionV3(include_top=False, pooling='avg',weights='imagenet')) # add a pre trained cnn to your network 
model.add(Flatten())

model.add(BatchNormalization())
model.add(Dense(2048 , activation='relu'))
model.add( BatchNormalization() )

model.add(Dense(1024,activation='relu'))
model.add( BatchNormalization() )
model.add(Dense(1,activation='sigmoid'))
model.layers[0].trainable=False

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(train_generator,
          steps_per_epoch=len(train_generator.filenames)//batch_size,
          epochs=4,
          validation_data=validation_generator,validation_steps=len(train_generator.filenames)//batch_size) 



#Test Model

In [ ]:
img_path='/content/male-female-face-dataset/Validation/female/112944.jpg.jpg'
img=load_img(img_path,target_size=(size,size,3))
plt.imshow(img)


In [ ]:
img=np.asarray(img)

In [ ]:
img=img/255.0

In [ ]:
img=img.reshape(1,size,size,3)

In [ ]:
img.shape

In [ ]:
model.predict(img)

In [ ]:
train_generator.class_indices

In [ ]:
def get_classes(data):
  prob=model.predict(img)
  pred=np.argmax(prob, axis=1)
  if pred[0]==0:
    return 'female' , prob[0][0]
  else :
      return 'male' , prob[0][0]

In [ ]:
get_classes(img)

# Real_Time Prediction using webcam

In [ ]:
import imutils
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

*Start webcam*

In [ ]:
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

**Click 'Capture' to make photo using your webcam.**

In [ ]:
image_file = take_photo()

*Read, resize and display the image*

In [ ]:
#image = cv2.imread(image_file, cv2.IMREAD_UNCHANGED)
image = cv2.imread(image_file)

# resize it to have a maximum width of 400 pixels
image = imutils.resize(image, width=400)
(h, w) = image.shape[:2]
print(w,h)
cv2_imshow(image)

In [ ]:
def get_prediction(img_path):
  img=load_img(img_path,target_size=(size,size,3))
  plt.imshow(img)
  img=img_to_array(img)
  img=img/255
  img.reshape(1,size,size,3)
  pred , prob =get_classes(img)
  return pred , prob 

In [ ]:
get_prediction('/content/photo.jpg')